In [ ]:
# Interactive Feynman Path Integral Notebook

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, VBox, Output
from IPython.display import display, clear_output

# Parameters
n_paths = 1000
n_steps = 500
T = 1.0
dt = T / n_steps
x_a = 0.0
x_b_values = np.linspace(-4, 4, 100)
slit_centers = [-1.0, 1.0]  # Centers of slits
slit_sigma = 0.2           # Width of slits
mass = 0.05
hbar = 1.0

# Time vector
time = np.linspace(0, T, n_steps + 1)

# Simulate the full interference pattern and store amplitudes and slit info
intensity_curve = []
all_amplitudes = []
all_slit_indices = []

for x_b in x_b_values:
    amplitudes = []
    slit_indices = []
    for _ in range(n_paths):
        x = np.zeros(n_steps + 1)
        x[0], x[-1] = x_a, x_b
        x[1:-1] = np.random.normal(loc=0.0, scale=np.sqrt(dt), size=n_steps - 1).cumsum()
        x[1:-1] += (x_b - x_a) * time[1:-1]  # Linear drift

        mid_index = n_steps // 2
        x_mid = x[mid_index]
        slit_index = np.argmin([abs(x_mid - center) for center in slit_centers])
        V = np.exp(-0.5 * ((x_mid - slit_centers[slit_index]) / slit_sigma) ** 2)

        S = 0.5 * mass * np.sum(np.diff(x) ** 2) / dt
        amplitude = np.exp(1j * S / hbar) * V
        amplitudes.append(amplitude)
        slit_indices.append(slit_index)

    psi = np.sum(amplitudes)
    intensity_curve.append(np.abs(psi) ** 2)
    all_amplitudes.append(amplitudes)
    all_slit_indices.append(slit_indices)

# Interactive output
out = Output()

def visualize_all(x_b):
    with out:
        clear_output(wait=True)

        index = np.abs(x_b_values - x_b).argmin()
        selected_amplitudes = all_amplitudes[index]
        selected_slits = all_slit_indices[index]

        fig, axs = plt.subplots(3, 1, figsize=(8, 14))

        # 1. Path Visualization
        highlighted_paths = []
        regular_paths = []
        for _ in range(n_paths):
            x = np.zeros(n_steps + 1)
            x[0], x[-1] = x_a, x_b
            x[1:-1] = np.random.normal(loc=0.0, scale=np.sqrt(dt), size=n_steps - 1).cumsum()
            x[1:-1] += (x_b - x_a) * time[1:-1]

            mid_index = n_steps // 2
            x_mid = x[mid_index]
            if any(np.abs(x_mid - center) < slit_sigma for center in slit_centers):
                highlighted_paths.append(x)
            else:
                regular_paths.append(x)

        for path in regular_paths:
            axs[0].plot(time, path, color='gray', alpha=0.05)
        for path in highlighted_paths:
            axs[0].plot(time, path, color='blue', alpha=0.2)

        axs[0].set_title(f"Sample Paths from x_a = {x_a} to x_b = {x_b:.2f}")
        axs[0].set_xlabel("Time")
        axs[0].set_ylabel("Position")
        axs[0].axvline(time[n_steps // 2], color='black', linestyle='--', linewidth=1)
        for center in slit_centers:
            axs[0].axhline(center, xmin=0.48, xmax=0.52, color='purple', linestyle='--', linewidth=2)

        # 2. Interference Pattern
        axs[1].plot(x_b_values, intensity_curve, label="Interference Pattern")
        axs[1].axvline(x_b, color='black', linestyle='--', label=f"x_b = {x_b:.2f}")
        axs[1].set_title("Interference Pattern with Slider Marker")
        axs[1].set_xlabel("Final Screen Position x_b")
        axs[1].set_ylabel(r"Probability Density |$\psi(x_b)|^2$")
        axs[1].legend()
        axs[1].grid(True)

        # 3. Phasor Diagram
        total_amplitude = np.sum(selected_amplitudes)
        axs[2].set_title(f"Phasor Sum at x_b = {x_b:.2f}")
        axs[2].set_xlabel("Re[Amplitude]")
        axs[2].set_ylabel("Im[Amplitude]")
        axs[2].grid(True)

        slit_colors = ['blue', 'orange']
        for amp, slit_idx in zip(selected_amplitudes, selected_slits):
            axs[2].arrow(0, 0, np.real(amp), np.imag(amp), alpha=0.1, color=slit_colors[slit_idx], head_width=0.0, head_length=0.0)

        axs[2].arrow(0, 0, np.real(total_amplitude), np.imag(total_amplitude), color='black', linewidth=2, label="Total Amplitude")
        axs[2].legend(handles=[
            plt.Line2D([0], [0], color='blue', lw=4, label='Slit 1'),
            plt.Line2D([0], [0], color='orange', lw=4, label='Slit 2'),
            plt.Line2D([0], [0], color='black', lw=2, label='Total Amplitude')
        ])

        zoom_range = 0.3
        axs[2].set_xlim(-zoom_range, zoom_range)
        axs[2].set_ylim(-zoom_range, zoom_range)

        plt.tight_layout()
        display(fig)
        plt.close(fig)

# Slider and Display
x_slider = FloatSlider(value=0.0, min=-4.0, max=4.0, step=0.1, description='x_b:')
ui = VBox([x_slider])

x_slider.observe(lambda change: visualize_all(change['new']), names='value')

display(ui, out)
visualize_all(x_slider.value)

Output()